# Multi Layer Perceptron Implementation

## Download the Data

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# Downlaod the Dataset 
taiwanese_bankruptcy_prediction = fetch_ucirepo(id=572) 
  
# Get the features and target variables
X = taiwanese_bankruptcy_prediction.data.features 
y = taiwanese_bankruptcy_prediction.data.targets 
  


## Pre-process the data

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Preprocess the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Build the Model

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.metrics import Precision, Recall


model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 635us/step - accuracy: 0.6919 - loss: 276758528.0000 - precision_10: 0.0403 - recall_10: 0.3366 
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.9503 - loss: 13287351.0000 - precision_10: 0.1180 - recall_10: 0.0661     
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.9425 - loss: 6195688.5000 - precision_10: 0.0409 - recall_10: 0.0358      
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.9419 - loss: 5059177.0000 - precision_10: 0.0889 - recall_10: 0.0862      
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.9396 - loss: 4038111.0000 - precision_10: 0.1042 - recall_10: 0.0935     
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.9490 - loss: 3526640.0000 - precision_10: 0.1028 - recall_10: 0.0790     
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step - accuracy: 0.9444 - loss: 3274077.7500 - precision_10: 0.0925 - recall_10: 0.0760     
Epoch

## Evaluate the Model

In [26]:

# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.9443 - loss: 6398529.5000 - precision_10: 0.0578 - recall_10: 0.0509      
Test loss: 5685806.5, Test accuracy: 0.9457477927207947, Precision: 0.0810810774564743, Recall: 0.06976744532585144
